In [ ]:
# 必要なものをインストール
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pyocr
!pip install beautifulsoup4

# 必要なモジュールをインポート
from PIL import Image
from bs4 import BeautifulSoup
import sys
import pyocr
import pyocr.builders
import time
import requests
import time

# Tesseractが正常にインストールされていない場合の処理と正常にインストールできていた場合
tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("error tool is not found")
    sys.exit(1)

tool = tools[0]
print("toolname is %s" % (tool.get_name()))

# 言語の確認
languages = tool.get_available_languages()
print("Available Language is %s" % " , " .join(languages))
language = languages[0]
print("Language is %s" % (language))

# ファイルの入力
from google.colab import files
f = files.upload()
filename = list(f)[0]

# 読み込んだ画像を表示させる
import cv2
import matplotlib.pyplot as plt
from pylab import rcParams

img = cv2.imread(filename)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(15, 15))
plt.axis("off")
plt.imshow(img)

# 画像を文字に変換する
txt = tool.image_to_string(
    Image.open(filename),
    lang = "eng",
    builder = pyocr.builders.TextBuilder(tesseract_layout=6)
)

# 品詞に分けて名詞のみ抽出する
!pip install nltk
import nltk
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

morph = nltk.word_tokenize(txt)
pos = nltk.pos_tag(morph)
words = [x for (x,y) in pos if y in 'NN']

def search_mean(spell):
    search_word = "https://ejje.weblio.jp/content/" + spell
    url = requests.get(search_word)
    soup = BeautifulSoup(url.text, "html.parser")
    word_mean = soup.find(class_='content-explanation ej').get_text()
    print(word_mean)

for word in words:
    print(word)
    try:
        search_mean(word)
    except Exception:
        print("存在しない単語です")
    time.sleep(2)